# Accessing GHCN-D in Databricks

### This notebook provides a quick overview of accessing GHCN - Daily data from Azure Blob Storage URL in Databricks. It then demonstrates some examples of writing queries to interact with the data stored in tables and come up with visualizations.

[References for the GHCN-D metadata](https://github.com/awslabs/open-data-docs/tree/main/docs/noaa/noaa-ghcn)

### Requirements:
- Run `station_metadata_processing.ipynb` to have the table `station_metadata` in schema `ghcn`

#### 1. Add `start_year` and `end_year` parameters for job workflows usage

In [ ]:
dbutils.widgets.text("start_year", "")
dbutils.widgets.text("end_year", "")

#### 2. Accessing GHCN-D data
- Read GHCN-daily data from azure blob storage URL into pandas dataframe
- drop rows that have quality check issue
- improve date time format
- write it into `ghcn.ghcn_{year}`table


In [ ]:
import pandas as pd

new_columns = ['ID', 'Time', 'Element', 'Value', 'M-Flag', 'Q-Flag', 'S-Flag', 'OBS-Time']

for year in range(int(dbutils.widgets.get("start_year")), int(dbutils.widgets.get("end_year")) + 1):
    
    URL = f'https://ghcn.blob.core.windows.net/ghcn/csv/daily/by_year/{year}.csv'

    df = pd.read_csv(URL, names = new_columns)

    df.drop(df[df['Q-Flag'].notnull() == True].index, inplace=True)
    
    df['Time'] = pd.to_datetime(df.Time, format='%Y%m%d')
    
    spark.createDataFrame(df).write.mode("overwrite").saveAsTable(f"ghcn.ghcn_{year}")


#### Query and Process the results into `ghcn_pivot_{end_year}` table for dynamic dashboard usage

In [ ]:
# query the data in US and specific climate attributes
merge_df = spark.sql(f'select date(g.Time), g.Element, mean(g.Value) as Value\
                              from ghcn.ghcn_{dbutils.widgets.get("end_year")} g\
                              join ghcn.station_metadata s\
                              on g.ID = s.ID\
                              where s.FIPS == "US" and g.Element in ("PRCP", "SNOW", "SNWD", "TMAX", "TMIN", "TAVG", "AWND")\
                              group by g.Time, g.Element\
                              order by g.Time')

# using `pivot_table()` to reshape the result
merge_df = merge_df.toPandas()
pivot_df = merge_df.pivot_table(index = 'Time', columns = 'Element', values = 'Value').reset_index().rename_axis(None, axis=1)
spark.createDataFrame(pivot_df).write.mode("overwrite").saveAsTable(f'ghcn.ghcn_pivot_{dbutils.widgets.get("end_year")}')
pivot_df

#### 3. In SQL, join `ghcn.station_metadata` and `ghcn.ghcn_{year}` tables to query the results you are interested in
- Aggregated monthly precipitation data from weather stations in Central and South America

In [ ]:
%sql
SELECT DISTINCT month(g.Time) as Month, round(mean(g.Value) / 10, 2) as Precipitation
FROM ghcn.ghcn_${end_year} g
JOIN ghcn.station_metadata s
ON g.ID = s.ID 
WHERE s.Region IN ('Central America', 'South America') AND g.Element == 'PRCP'
GROUP BY Month
ORDER BY Month

- Aggregated daily average temperature data from weather stations in North America

In [ ]:
%sql
SELECT DISTINCT date(g.Time) as Date, round(mean(g.Value) / 10, 2) as AvgTemperature
FROM ghcn.ghcn_${end_year} g
JOIN ghcn.station_metadata s
ON g.ID = s.ID 
WHERE s.Region = 'North America' AND g.Element = 'TAVG'
GROUP BY Date
ORDER BY Date

- Temperature map in July `end_year` in the United States

In [ ]:
%sql
SELECT s.State, round(mean(g.Value) / 10, 2) as AvgTemperature
FROM ghcn.ghcn_${end_year} g
JOIN ghcn.station_metadata s
ON g.ID = s.ID 
WHERE s.FIPS == 'US' AND g.Element = 'TAVG' AND month(g.Time) == '5'
GROUP BY s.State
ORDER BY s.State

- Temperature map in Jan `end_year` in the world

In [ ]:
%sql
SELECT s.`ISO-2`, round(mean(g.Value) / 10, 2) as AvgTemperature
FROM ghcn.ghcn_${end_year} g
JOIN ghcn.station_metadata s
ON g.ID = s.ID 
WHERE g.Element = 'TAVG' AND month(g.Time) == '1'
GROUP BY s.`ISO-2`